In [6]:
import pandas as pd
import numpy as np
import re
from datetime import datetime

def processar_planilhas():
    recebimento = pd.read_excel('recebimento_ajustado.xlsx')
    expedicao = pd.read_excel('expedicao.xlsx')
    lista_produtos = pd.read_excel('listadeprodutos.xlsx')
    matriz_distancias = pd.read_excel('matriz_distancias.xlsx', index_col=0)

    expedicao = expedicao.rename(columns={
        'Nº da ordem de transporte': 'ordem',
        'Material': 'material',
        'Qtd.teórica origem': 'quantidade',
        'Posição dep.origem': 'origem',
        'Posição de destino enclausuramentos': 'destino',
        'Data de criação': 'data',
        'Hora da criação': 'hora'
    })

    recebimento = recebimento.rename(columns={
        'Nº da ordem de transporte': 'ordem',
        'tipo de produto': 'material',
        'Qtd.teórica origem': 'quantidade',
        'posição de origem ': 'origem',
        'Posição dep.destino': 'destino',
        'Data de criação': 'data',
        'Hora da criação': 'hora'
    })

    def padronizar_localizacao(loc):
      if pd.isna(loc):
          return loc
      loc = str(loc)

      if loc.isdigit():
          return f"e{loc.zfill(2)}"

      elif re.match(r'^A[VX][A-Za-z0-9]+$', loc):
          return loc.upper().strip()

      elif loc.startswith(('Esteira', 'E')):
          return loc

      else:
          return loc.upper().strip()

    expedicao['destino'] = expedicao['destino'].apply(padronizar_localizacao)
    recebimento['origem'] = recebimento['origem'].apply(padronizar_localizacao)
    expedicao['operacao'] = 'expedicao'
    recebimento['operacao'] = 'recebimento'

    df_unificado = pd.concat([recebimento, expedicao], ignore_index=True)

    df_unificado['data_hora'] = pd.to_datetime(
        df_unificado['data'].astype(str) + ' ' + df_unificado['hora'].astype(str))

    lista_produtos = lista_produtos.rename(columns={
        'PNC': 'material',
        'Base Bloco': 'base'
    })
    df_unificado = pd.merge(df_unificado, lista_produtos[['material', 'base']],
                           on='material', how='left')

    def calcular_distancia(row):
        origem = row['origem']
        destino = row['destino']

        if origem in matriz_distancias.columns and destino in matriz_distancias.index:
            return matriz_distancias.loc[destino, origem]
        return np.nan

    df_unificado['distancia'] = df_unificado.apply(calcular_distancia, axis=1)

    df_unificado = df_unificado.sort_values('data_hora').reset_index(drop=True)
    colunas_finais = [
        'operacao', 'ordem', 'material', 'base', 'quantidade',
        'origem', 'destino', 'data_hora', 'distancia'
    ]
    df_unificado = df_unificado[colunas_finais]
    df_unificado.to_excel('ordens_unificadas.xlsx', index=False)
    print("Planilha criada")

    return df_unificado

df_final = processar_planilhas()

Planilha unificada criada com sucesso!
